### tried sgd (poor o/p)

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [4]:
# Importing Basic libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns

from collections import Counter
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix, roc_auc_score,f1_score


from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
# Importing libraries for running different models

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

In [6]:
# Libraries for text preprocessing
import re
import nltk

#nltk.download('stopwords')
from nltk.corpus import stopwords # import stopwords 

#nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer # to lemmatize the words

from nltk.tokenize import RegexpTokenizer, word_tokenize # to tokenize sentences into words

### Loading Train dataset

In [7]:
data = pd.read_csv("E:\\1 P36 PROJECT\\Original Datasets\\train (2).csv")

In [32]:
data.shape
data.head(3) # (14343, 3)

(14343, 4)

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   
2   2  great location great location 5 mins subway ta...       5   

                                        Clean_review  
0  exceptional service nice around daughter staye...  
1  beautiful relaxing jw marriott desert ridge re...  
2  great location great location min subway take ...

## Preprocessing of train data

In [9]:
# Replacing contractions (shorter vesions of words) with full word (n't , 'nt  as not)
for index, row in data.iterrows():
    phrase = row['Review']
    phrase = phrase.lower() #Convert to lowercase
    phrase = re.sub(r"n\'t", " not", phrase) # replace n't  with not
    phrase = re.sub(r"\'nt", " not", phrase) # replace 'nt  with not 
    data.at[index, 'Review_2'] = phrase
data[0:3]

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   
2   2  great location great location 5 mins subway ta...       5   

                                            Review_2  
0  exceptional service nice all-around daughter s...  
1  beautiful relaxing jw marriott desert ridge re...  
2  great location great location 5 mins subway ta...

In [10]:
# TEXT NORMALIZATION   or PRE-PROCESSING

##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english")) -set(["not"])
##Creating a list of custom stopwords
new_words = ['n',"th","hotel",'hotels','room','rooms','restaurant','restaurants','e','ca','nd','wo','el','etc']
stop_words = stop_words.union(new_words)

for index, row in data.iterrows():
    text = re.sub('[^a-zA-Z]', ' ', row['Review_2']) # Remove punctuations    
    # text = text.lower() #Convert to lowercase
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)  # remove tags
    text = re.sub("(\\d|\\W)+"," ",text)  # remove special characters and digits
    text = text.split()  # Convert to list from string
    
    #Lemmatisation
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text if not word  in stop_words] 
    text_string = " ".join(text)
    data.at[index, 'Clean_review'] = text_string
data.head(3)

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   
2   2  great location great location 5 mins subway ta...       5   

                                            Review_2  \
0  exceptional service nice all-around daughter s...   
1  beautiful relaxing jw marriott desert ridge re...   
2  great location great location 5 mins subway ta...   

                                        Clean_review  
0  exceptional service nice around daughter staye...  
1  beautiful relaxing jw marriott desert ridge re...  
2  great location great location min subway take ...

In [11]:
data.drop(['Review_2'], axis=1,inplace=True)

## TFIDF for train data

In [12]:
# get clean-review text column as list
clean_review_list = data['Clean_review'].tolist()
type(clean_review_list)
clean_review_list[0]

list

'exceptional service nice around daughter stayed well priced night king suite weekend night really impressed exceptionally welcoming service time reservation departure employee talked really friendly eager help question amazed day age l shaped pool tiny looked like fun kid hot tub exercise well breakfast abundant make waffle add scrambled egg bacon sausage fruit available bagel muffin oatmeal yogurt wanted morning sort long line waiting make waffle waiting scrambled egg picky really nice asked away freeway reading review earlier bit noisy hey city felt like deluxe considering reasonable price nice bedding art wall really liked space needle bit window shuttle run hourly not reserve certain place certain time worked shuttle missed minute needed return space needle promptly paid cab fare asked definitely need stay near downtown seattle'

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(min_df=750)
# tfidfconverter = TfidfVectorizer(max_features=3000)
tfidf_all = tfidfconverter.fit_transform(clean_review_list).toarray()

tfidf_all_names = tfidfconverter.get_feature_names() ; tfidf_all_names[:5]

tfidf_all.shape 
# there are 14343 rows/documents and 429 terms have been generated

tfidf_all_vector =pd.DataFrame(tfidf_all, columns=tfidf_all_names)
tfidf_all_vector.head(3)
# 3000 terms 

['able', 'absolutely', 'access', 'actually', 'air']

(14343, 280)

able  absolutely  access  actually  air   airport  amazing  area  arrival  \
0   0.0         0.0     0.0       0.0  0.0  0.000000      0.0   0.0      0.0   
1   0.0         0.0     0.0       0.0  0.0  0.146389      0.0   0.0      0.0   
2   0.0         0.0     0.0       0.0  0.0  0.000000      0.0   0.0      0.0   

   arrived  ...       way  week   weekend  went  wife    window  wonderful  \
0      0.0  ...  0.000000   0.0  0.138406   0.0   0.0  0.131637    0.00000   
1      0.0  ...  0.000000   0.0  0.000000   0.0   0.0  0.000000    0.26124   
2      0.0  ...  0.145193   0.0  0.000000   0.0   0.0  0.000000    0.00000   

   work  worth      year  
0   0.0    0.0  0.000000  
1   0.0    0.0  0.141784  
2   0.0    0.0  0.000000  

[3 rows x 280 columns]

### target variable

In [14]:
y = data.Rating ; type(y) ; y.shape

pandas.core.series.Series

(14343,)

## Test data

In [15]:
data_test = pd.read_csv("E:\\1 P36 PROJECT\\Original Datasets\\test (2).csv")

In [16]:
data_test.shape
data_test.head(3)
# 6148 and 2 

(6148, 2)

ID                                             Review
0   0  just superb rendezvous just perfect property s...
1   1  better close staten island ferry easy subway, ...
2   2  enjoyed stay, just come long weekend barcelona...

## Preprocessing of test data

In [17]:
# Replacing contractions (shorter vesions of words) with full word (n't , 'nt  as not)
for index, row in data_test.iterrows():
    phrase = row['Review']
    phrase = phrase.lower() #Convert to lowercase
    phrase = re.sub(r"n\'t", " not", phrase) # replace n't  with not
    phrase = re.sub(r"\'nt", " not", phrase) # replace 'nt  with not 
    data_test.at[index, 'Review_2'] = phrase
data_test[0:3]

ID                                             Review  \
0   0  just superb rendezvous just perfect property s...   
1   1  better close staten island ferry easy subway, ...   
2   2  enjoyed stay, just come long weekend barcelona...   

                                            Review_2  
0  just superb rendezvous just perfect property s...  
1  better close staten island ferry easy subway, ...  
2  enjoyed stay, just come long weekend barcelona...

In [18]:
# TEXT NORMALIZATION   or PRE-PROCESSING

##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english")) -set(["not"])
##Creating a list of custom stopwords
new_words = ['n',"th","hotel",'hotels','room','rooms','restaurant','restaurants','e','ca','nd','wo','el','etc']
stop_words = stop_words.union(new_words)

for index, row in data_test.iterrows():
    text = re.sub('[^a-zA-Z]', ' ', row['Review_2']) # Remove punctuations    
    # text = text.lower() #Convert to lowercase
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)  # remove tags
    text = re.sub("(\\d|\\W)+"," ",text)  # remove special characters and digits
    text = text.split()  # Convert to list from string
    
    #Lemmatisation
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text if not word  in stop_words] 
    text_string = " ".join(text)
    data_test.at[index, 'Clean_review'] = text_string
data_test.head(3)

ID                                             Review  \
0   0  just superb rendezvous just perfect property s...   
1   1  better close staten island ferry easy subway, ...   
2   2  enjoyed stay, just come long weekend barcelona...   

                                            Review_2  \
0  just superb rendezvous just perfect property s...   
1  better close staten island ferry easy subway, ...   
2  enjoyed stay, just come long weekend barcelona...   

                                        Clean_review  
0  superb rendezvous perfect property singapore f...  
1  better close staten island ferry easy subway f...  
2  enjoyed stay come long weekend barcelona staye...

In [19]:
data_test.drop(['Review_2'], axis=1,inplace=True)

## TFIDF for Test data

In [20]:
# get clean-review text column as list
clean_review_list_test = data_test['Clean_review'].tolist()
type(clean_review_list_test)
clean_review_list_test[0]

list

'superb rendezvous perfect property singapore fourth time family stayed property clean staff welcoming standard service second none stayed york terrible westin stamford excellent pricey rendezvous stack count perfect location right not better return therendezvous'

#### finding the terms in test data

In [21]:
clean_review_words_test = " ".join(data_test['Clean_review'])
clean_review_words_test = clean_review_words_test.split()
len(clean_review_words_test)
type(clean_review_words_test)
 # 5,72,663 words are present in cleaned reviews of test data (includes duplicate words as well)

572663

list

In [22]:

# finding frequency of all words in test data

words_all = len(clean_review_words_test)
words_all # 572663

word_dist = nltk.FreqDist(clean_review_words_test)

freq_data_all_test = pd.DataFrame(word_dist.most_common(words_all), columns=['Word','Frequency'])

freq_data_all_test['percentFreq_all'] =(freq_data_all_test.Frequency / freq_data_all_test.Frequency.sum())*100 
freq_data_all_test['percentFreq_unique'] =(freq_data_all_test.Frequency / len(freq_data_all_test))*100 

len(freq_data_all_test) # 23896
freq_data_all_test[0:10]

words_all_1= freq_data_all_test.Frequency.sum() 
words_all_1 # 1368889

# word "not" has not been removed from our list 
# we have total 572663  words
# we have 23878 (23896) unique words 

572663

23896

Word  Frequency  percentFreq_all  percentFreq_unique
0       not      15164         2.647980           63.458319
1     great       6288         1.098028           26.314027
2      good       5204         0.908737           21.777703
3     staff       5006         0.874162           20.949113
4      stay       4716         0.823521           19.735521
5     night       4251         0.742321           17.789588
6       day       3898         0.680680           16.312354
7      nice       3810         0.665313           15.944091
8      time       3773         0.658852           15.789253
9  location       3421         0.597385           14.316204

572663

In [35]:
# Using the train data transformer, we will transform test data

tfidf_test = tfidfconverter.transform(clean_review_list_test).toarray()

# using feature names that were already obtained from train data 
# tfidf_all_names = tfidfconverter.get_feature_names()  
tfidf_all_names[:5]

tfidf_test.shape 

tfidf_test_vector =pd.DataFrame(tfidf_test, columns=tfidf_all_names)
tfidf_test_vector.head(3)
# 280 terms 

['able', 'absolutely', 'access', 'actually', 'air']

(6148, 280)

able  absolutely  access  actually  air  airport  amazing  area  \
0  0.000000         0.0     0.0  0.000000  0.0    0.000      0.0   0.0   
1  0.000000         0.0     0.0  0.163821  0.0    0.000      0.0   0.0   
2  0.115952         0.0     0.0  0.000000  0.0    0.103      0.0   0.0   

   arrival   arrived  ...  way  week   weekend  went  wife  window  wonderful  \
0      0.0  0.000000  ...  0.0   0.0  0.000000   0.0   0.0     0.0        0.0   
1      0.0  0.000000  ...  0.0   0.0  0.000000   0.0   0.0     0.0        0.0   
2      0.0  0.101134  ...  0.0   0.0  0.118773   0.0   0.0     0.0        0.0   

   work  worth  year  
0   0.0    0.0   0.0  
1   0.0    0.0   0.0  
2   0.0    0.0   0.0  

[3 rows x 280 columns]

In [36]:
data.head(2)
data_test.head(2)

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   

                                        Clean_review  
0  exceptional service nice around daughter staye...  
1  beautiful relaxing jw marriott desert ridge re...

ID                                             Review  \
0   0  just superb rendezvous just perfect property s...   
1   1  better close staten island ferry easy subway, ...   

                                        Clean_review  
0  superb rendezvous perfect property singapore f...  
1  better close staten island ferry easy subway f...

### Training SVC on same data by splitting it (80:20) , 280 terms

In [37]:
# # Splitting dataset in 80:20 ratio
# frpm previous models. found this ratio to give good results
X_train1,X_test1,y_train1,y_test1 = train_test_split(tfidf_all_vector,y,test_size=0.20,random_state=123,stratify=y)
X_train1.shape # (11474, 3000)
X_test1.shape # (2869, 3000)
y.shape  # 14343 reviews

(11474, 280)

(2869, 280)

(14343,)

In [38]:
##### Running the model on train data and validating on validation data

model_svm= SVC(random_state=123)
model_svm.fit(X_train1,y_train1)


# for training data
y_pred_train = model_svm.predict(X_train1)

print("Accuracy score on train data:" ,accuracy_score(y_train1,y_pred_train))
print("F1 macro score on train data:",f1_score(y_train1,y_pred_train, average='macro'))
print(classification_report(y_train1,y_pred_train))
print(confusion_matrix(y_train1,y_pred_train))
# plot_confusion_matrix(model_log, X_train1,y_train1,cmap=plt.cm.Blues); plt.show();

# for test data 
y_pred_test = model_svm.predict(X_test1)

print("Accuracy score on test data:" ,accuracy_score(y_test1,y_pred_test))
print("F1 macro score on test data:",f1_score(y_test1,y_pred_test, average='macro'))
print(classification_report(y_test1,y_pred_test))
print(confusion_matrix(y_test1,y_pred_test))

# plotting confusion matrix
# plot_confusion_matrix(model_log, X_test1,y_test1,cmap=plt.cm.Blues); plt.show();

SVC(random_state=123)

Accuracy score on train data: 0.8581139968624717
F1 macro score on train data: 0.8399062316475788
              precision    recall  f1-score   support

           1       0.92      0.89      0.90       782
           2       0.88      0.81      0.84       998
           3       0.93      0.61      0.74      1208
           4       0.83      0.80      0.81      3337
           5       0.86      0.96      0.90      5149

    accuracy                           0.86     11474
   macro avg       0.88      0.81      0.84     11474
weighted avg       0.86      0.86      0.85     11474

[[ 696   34    8   19   25]
 [  31  806   24   78   59]
 [  21   49  737  264  137]
 [   7   25   13 2682  610]
 [   4    5    8  207 4925]]
Accuracy score on test data: 0.5907981875217846
F1 macro score on test data: 0.4821751813563898
              precision    recall  f1-score   support

           1       0.61      0.61      0.61       195
           2       0.39      0.34      0.36       250
           3 

## SVC on train data  and evaluating test data

In [28]:
tfidf_all_vector.shape # (14343, 3006)
tfidf_test_vector.shape # (6148, 3006)

(14343, 3000)

(6148, 3000)

In [39]:
### SGD on train and evaluate test data

model_svm= SVC(random_state=123)
model_svm.fit(tfidf_all_vector,y)

# for training data
y_pred_train = model_svm.predict(tfidf_all_vector)

print("Accuracy score :" ,accuracy_score(y,y_pred_train))
print("F1 macro score :",f1_score(y,y_pred_train, average='macro'))
print(classification_report(y,y_pred_train))
print(confusion_matrix(y,y_pred_train))

# for test data 
test_pred = model_svm.predict(tfidf_test_vector)

# Multinomial Logistic Regression on tfidf using max_featr=3000 terms, all 6 new features
# accuracy of train data is 94% and F1-macro score is 94%

SVC(random_state=123)

Accuracy score : 0.851844105138395
F1 macro score : 0.8312375990374171
              precision    recall  f1-score   support

           1       0.89      0.89      0.89       977
           2       0.87      0.79      0.82      1248
           3       0.92      0.61      0.73      1510
           4       0.82      0.79      0.81      4172
           5       0.85      0.95      0.90      6436

    accuracy                           0.85     14343
   macro avg       0.87      0.81      0.83     14343
weighted avg       0.85      0.85      0.85     14343

[[ 873   45   15   20   24]
 [  58  981   40   93   76]
 [  34   62  923  321  170]
 [  12   32   15 3308  805]
 [   7   11   12  273 6133]]


In [40]:
type(test_pred)
test_pred.shape
test_pred[0:5]

# converting pred values to df

df_pred = pd.DataFrame(test_pred, columns=['Rating'])
df_pred.shape

data_test.columns
df_pred["ID"] = data_test["ID"]

col_titles = ["ID","Rating"]
df_pred = df_pred.reindex(columns=col_titles)
df_pred.head()

numpy.ndarray

(6148,)

array([5, 3, 4, 5, 4], dtype=int64)

(6148, 1)

Index(['ID', 'Review', 'Clean_review'], dtype='object')

ID  Rating
0   0       5
1   1       3
2   2       4
3   3       5
4   4       4

In [41]:
# Exporting to csv file
df_pred.to_csv(r'E:\1 P36 PROJECT\Predictions\df_pred_svc2_280.csv',index=False)